In [13]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory, TerminationCondition
import psycopg2

## Conectar Base de datos y Preparacion de los datos a utilizar

In [14]:
conexion = psycopg2.connect(
        dbname="buap",
        user="postgres",
        password="contrasena",
        host="localhost",
        port="5432"
    )
cursor = conexion.cursor()
datos = {}

In [15]:
# Profesores (con nombre)
cursor.execute("SELECT id_profesor, nombre FROM Profesor LIMIT 50")
profesores = cursor.fetchall()
datos['Id_Profesores'] = [row[0] for row in profesores] # Lista de IDs de profesores
datos['nombres_profesor'] = {row[0]: row[1] for row in profesores} # Diccionario de nombres de profesores


In [16]:
# Materias (con nombre)
cursor.execute("SELECT id_materia, nombre, horas_por_semana, id_tipo_clase FROM Materia LIMIT 50")
materias = cursor.fetchall()
datos['id_Materias'] = [row[0] for row in materias] # Lista de IDs de materias
datos['nombres_materia'] = {row[0]: row[1] for row in materias} # Diccionario de nombres de materias
datos['Horas_Semana_Materia'] = {row[0]: row[2] for row in materias} # Diccionario de horas por semana de cada materia
datos['Tipo_Clase'] = {row[0]: row[3] for row in materias} # Diccionario de tipo de clase de cada materia

In [17]:
# Aulas
cursor.execute("SELECT id_aula, id_edificio, id_tipo_clase FROM Aula")
rows = cursor.fetchall()

datos['Aulas'] = [(row[0], row[1]) for row in rows] # Lista de IDs compuestos (id_aula, id_edificio)
datos['Tipo_Aula'] = {(row[0], row[1]): row[2] for row in rows}# Diccionario: clave compuesta (id_aula, id_edificio) -> tipo clase


# Bloques horarios
datos['Dia'] = list(range(1, 6)) # Lunes a Viernes
datos['Hora'] = list(range(1, 11)) # 10 bloques horarios por día/ de 8 am a 6 pm

In [18]:
aulas_param = datos['Aulas']  # lista de tuplas (id_aula, id_edificio)

query = """
SELECT A.id_aula, A.id_edificio, M.id_materia,
       CASE WHEN A.id_tipo_clase = M.id_tipo_clase THEN 1 ELSE 0 END AS compatible,
       M.id_tipo_clase AS tipo_materia,
       A.id_tipo_clase AS tipo_aula
FROM Aula A
CROSS JOIN Materia M
WHERE (A.id_aula, A.id_edificio) = ANY(%s) AND M.id_materia = ANY(%s)
"""

cursor.execute(query, (aulas_param, datos['id_Materias']))

datos['Compatibiliadad_Aula_Materia'] = {
    ((row[0], row[1]), row[2]): {
        'compatible': row[3],
        'tipo_materia': row[4],
        'tipo_aula': row[5]
    }
    for row in cursor.fetchall()
}


In [19]:
from collections import defaultdict
datos['Materias_por_Profesor'] = defaultdict(list)
datos['Profesores_por_Materia'] = defaultdict(list)
datos['Profesores_Imaginarios'] = []
datos['Profesores_Totales'] = []

cursor.execute("""
    SELECT id_profesor, id_materia 
    FROM profesor_materia
    WHERE id_profesor = ANY(%s) AND id_materia = ANY(%s)
""", (datos['Id_Profesores'], datos['id_Materias']))

for profe_id, materia_id in cursor.fetchall():
    datos['Materias_por_Profesor'][profe_id].append(materia_id)
    datos['Profesores_por_Materia'][materia_id].append(profe_id)

materias_sin_profesor = [m for m in datos['id_Materias'] if m not in datos['Profesores_por_Materia']]

profesores_imaginarios = [f"PA_{i}" for i in range(len(materias_sin_profesor))]
datos['Profesores_Imaginarios'] = profesores_imaginarios
print("Se han creado", len(profesores_imaginarios), "profesores imaginarios para las materias sin profesor asignado.")
#Asignación de profesores imaginarios
for i, materia_id in enumerate(materias_sin_profesor):
    profe_imag = profesores_imaginarios[i]
    datos['Profesores_por_Materia'][materia_id].append(profe_imag)
    datos['Materias_por_Profesor'][profe_imag].append(materia_id)

datos['Profesores_Totales'].extend(datos['Id_Profesores'])
datos['Profesores_Totales'].extend(profesores_imaginarios)

datos['nombres_profesor'].update({
    p: f"ProfFicticio_{i}" for i, p in enumerate(profesores_imaginarios)
})

Se han creado 29 profesores imaginarios para las materias sin profesor asignado.


In [20]:
datos['Materia_Sesion_Minima'] = {id_materia: 1 for id_materia in datos['id_Materias']}
datos['Materia_Sesion_Maxima'] = {id_materia: 3 for id_materia in datos['id_Materias']}
datos['Hora_Minima_Profesor'] = {id_prof: 0 for id_prof in datos['Profesores_Totales']}
datos['Hora_Maxima_Profesor'] = {id_prof: 20 for id_prof in datos['Profesores_Totales']}

## Crear Modelo de Pyomo

---

### Conjuntos

- $P: \text{Profesores}$
    - $P_R: \text{Profesores Reales}$
    - $P_I: \text{Profesores Imaginarios}$
    - $P = P_R \cup\ P_I$
- $P(j):$ Profesores que pueden enseñar la materia $j$
    - $P(j) \subset P$
- $M: \text{Materias}$
- $M(i):$ Materias que puede dar el profesor $i$
    - $M(i) \subset M$
- $A: \text{Aulas}$
    - $A = A_{salon}\ \cup\ A_{edificio}$
- $BH: \text{Bloques Horarios}$
    - $D: \text{Dias}, D = \{1, 2,\ \dots, 5\}$
    - $H: \text{Horarios}, H = \{1, 2,\ \dots, 10\}$
    - $BH = D \times H = \{(d, h)\ |\ d \in D, h \in H\}$
- $T: \text{Tipo}$
    - $T = \{ 1, 2, 3, 4 \}$

In [21]:
# Definir los conjuntos
model = pyo.ConcreteModel()

def M_prof_init(model, i):
    return datos['Materias_por_Profesor'][i]

def P_mat_init(model, j):
    return datos['Profesores_por_Materia'][j]

# Conjuntos
model.ProfesoresReales = pyo.Set(initialize=datos['Id_Profesores'])
model.ProfesoresImaginarios = pyo.Set(initialize=datos['Profesores_Imaginarios'])
model.Profesores = pyo.Set(initialize=datos['Profesores_Totales'])
model.Materias = pyo.Set(initialize=datos['id_Materias'])
model.M_prof = pyo.Set(model.Profesores, initialize=M_prof_init)
model.P_mat = pyo.Set(model.Materias, initialize=P_mat_init)
model.Aulas = pyo.Set(dimen=2, initialize=[(aula_id, edificio_id) for aula_id, edificio_id in datos['Aulas']])
model.BloquesHorarios = pyo.Set(dimen=2, initialize=[(dia, hora) for dia in datos['Dia'] for hora in datos['Hora']])
model.Dias = pyo.Set(initialize=datos['Dia'])
model.Horas = pyo.Set(initialize=datos['Hora'])
model.Tipo = pyo.RangeSet(1, 4)

### Parametros

- $\tau(j):$ Horas a la semana del materia $j$
- $\sigma (j):$ Tipo de clase de la materia $j$
- $\sigma (a):$ Tipo de aula del aula $a$
- $\omega (a, j) = \begin{cases} 1 & \text{si el aula}\ a\ \text{es compatible con la materia}\ j\\ 0 & \text{si no} \end{cases}$

- $ \omega (i, j) = \begin{cases} 1 & \text{si el profesor}\ p\ \text{puede dar la materia}\ j\\ 0 & \text{si no} \end{cases}$
- $\psi_{min}(j):$ Duración mínima de la sesión del curso $j$
- $\psi_{max}(j):$ Duración máxima de la sesión del curso $j$
- $\gamma_{min}(i):$ Horas mínimas que puede impartir el profesor $i$
- $\gamma_{max}(i):$ Horas máximas que puede impartir el profesor $i$

In [22]:
# Evitar sobreescribir
for attr in ['tau', 'sigma_materia', 'sigma_aula', 'omega_aula_materia', 'omega_prof_materia', 'Psi_Min', 'Psi_Max']:
    if hasattr(model, attr):
        model.del_component(getattr(model, attr))

model.tau = pyo.Param(model.Materias, initialize=datos['Horas_Semana_Materia'])
model.sigma_materia = pyo.Param(model.Materias, initialize=datos['Tipo_Clase'])
model.sigma_aula = pyo.Param(model.Aulas, initialize=datos['Tipo_Aula'])

model.omega_aula_materia = pyo.Param(
    model.Aulas, model.Materias,
    initialize={
        (aula, materia): datos['Compatibiliadad_Aula_Materia'].get((aula, materia), {}).get('compatible', 0)
        for aula in model.Aulas for materia in model.Materias
    }
)

model.omega_prof_materia = pyo.Param(
    model.Profesores, model.Materias,
    initialize={
        (profe_id, materia_id): 1 if materia_id in datos['Materias_por_Profesor'][profe_id] else 0
        for profe_id in model.Profesores for materia_id in model.Materias
    }
)

model.Psi_Min = pyo.Param(model.Materias, initialize=datos['Materia_Sesion_Minima'])
model.Psi_Max = pyo.Param(model.Materias, initialize=datos['Materia_Sesion_Maxima'])
model.gamma_min = pyo.Param(model.Profesores, initialize=datos['Hora_Minima_Profesor'])
model.gamma_max = pyo.Param(model.Profesores, initialize=datos['Hora_Maxima_Profesor'])

### Variables

- $
x^{i, j}_{k, t} =
\begin{cases}
    1 & \text{si el profesor } i
    \text{ tiene asignada la materia } j
    \text{ en el aula } k
    \text{ en el horario } t \\
    0 & \text{si no}
\end{cases}
$

- $
y_{i, j} =
\begin{cases}
    1 & \text{si el profesor } i
    \text{ tiene asignada la materia } j \\
    0 & \text{si no}
\end{cases}
$

- $
z_{j, d} =
\begin{cases}
    1 & \text{si la materia } j
    \text{ se imparte el dia } d \\
    0 & \text{si no}
\end{cases}
$

- $
v_{k, j} =
\begin{cases}
    1 & \text{si el aula } k
    \text{ esta asignada a la materia } j \\
    0 & \text{si no}
\end{cases}
$

- $
s_{j, h, d} =
\begin{cases}
    1 & \text{si la materia } j
    \text{ comienza a la hora } h
    \text{ en el dia } d \\
    0 & \text{si no}
\end{cases}
$

In [23]:
# Lista de variables a gestionar
variables = ['X', 'Y', 'Z', 'V', 'S']

# Eliminar variables existentes para evitar sobreescribir
for var in variables:
    if hasattr(model, var):
        model.del_component(getattr(model, var))

# Definir las variables nuevamente

# X: Profesor, Materia, Aula, Bloque horario
X_index = [
    (p, m, a, bh)
    for p in datos['Profesores_Totales']  # Incluye reales e imaginarios
    for m in datos['id_Materias']
    if m in datos['Materias_por_Profesor'].get(p, [])  # Validación para evitar KeyError
    for a in datos['Aulas']
    if datos['Compatibiliadad_Aula_Materia'].get((a, m), {}).get('compatible', 0) == 1  # Aulas compatibles
    for bh in [(d, h) for d in datos['Dia'] for h in datos['Hora']]
]
model.X = pyo.Var(X_index, domain=pyo.Binary)

# Indices para componentes de X
PROFESOR = 0
MATERIA = 1
AULA = 2
EDIFICIO = 3
DIA = 4
HORA = 5

# X_Index 
#(100004288, 1, (102, 5), (1, 2))
# Model.X tiene la siguiente estructura para sus indices, pyomo aplana las tuplas :
# (100004288, 1, 102, 5, 1, 1) 
# Y: Profesor asignado a materia (solo combinaciones posibles)

Y_index = [(i, j) for i in datos['Profesores_Totales'] for j in datos['Materias_por_Profesor'].get(i, [])]
model.Y = pyo.Var(Y_index, domain=pyo.Binary)

# Z: Materia impartida en día d
model.Z = pyo.Var(model.Materias, model.Dias, domain=pyo.Binary)

# V: Aula asignada a materia (solo combinaciones compatibles)
V_index = [
    (aula, edificio, materia)
    for ((aula, edificio), materia), val in datos['Compatibiliadad_Aula_Materia'].items()
    if val['compatible'] == 1
]
model.V = pyo.Var(V_index, domain=pyo.Binary)

# S: Materia j en hora h y día d
model.S = pyo.Var(model.Materias, model.Horas, model.Dias, domain=pyo.Binary)

# Logs para reducción de variables
print(f"X: Variables reducidas de {len(datos['Profesores_Totales'])*len(datos['id_Materias'])*len(datos['Aulas'])*len(datos['Dia'])*len(datos['Hora'])} a {len(X_index)}")
print(f"Y: Variables reducidas de {len(datos['Profesores_Totales'])*len(datos['id_Materias'])} a {len(Y_index)}")

X: Variables reducidas de 23107500 a 200250
Y: Variables reducidas de 3950 a 63


### Restricciones

- El curso $j$ debe tener asignado un profesor (No hace falta checar compatibilidad, ya que Y tiene solo valores posibles):
$$
\sum_{i \in P} y_{i, j} = 1,\ \forall\ j \in M
$$

In [24]:
#Funciona Todas las materias tiene profesor
if hasattr(model, 'Restriccion_Materia_Tiene_Profe'):
    model.del_component(model.Restriccion_Materia_Tiene_Profe)

def materia_tiene_profe_rule(model, j):
    return sum(
        model.Y[idy]
        for idy in Y_index
        if idy[1] == j
    ) == 1

model.Restriccion_Materia_Tiene_Profe = pyo.Constraint(model.Materias, rule=materia_tiene_profe_rule)

- Todas las horas de la materia $j$ deben estar cubiertas, y el profesor $i$ debe impartir todas las horas del curso $j$, si el es seleccionada para impartirlo:

$$
\sum_{k \in A} \sum_{t \in BH} x^{i, j}_{k, t} = \tau(j) \cdot y_{i, j} \quad \forall\ j \in M, i \in P(j)
$$

In [25]:
#Funciona Se da el numero total de horas semanales requeridas por cada materia
if hasattr(model, 'Restriccion_Profesor_Cubre_Curso'):
    model.del_component(model.Restriccion_Profesor_Cubre_Curso)

# X_by_prof_materia = {}
# for key in model.X:
#     i, j, _, _, _, _ = key  # (profesor, materia, aula, (d,h))
#     X_by_prof_materia.setdefault((i, j), []).append(key)

def profesor_cubre_curso_rule(model, i, j):
    # keys = X_by_prof_materia.get((i, j), [])
    # if not keys:  # If this professor can't teach this subject, skip
    #     return pyo.Constraint.Skip
    return sum(
        model.X[idx]
        for idx in X_index
        if idx[PROFESOR] == i
        and idx[MATERIA] == j
    ) == model.tau[j] * model.Y[i, j]

model.Restriccion_Profesor_Cubre_Curso = pyo.Constraint(((i, j) for j in model.Materias for i in model.P_mat[j]), rule=profesor_cubre_curso_rule)

- No Solapamiento Aula (No asignar un aula a más de una materia en un bloque horario):
$$
\sum_{i \in P} \sum_{j \in M} x^{i, j}_{k, t} \le 1 \quad \forall\ k \in A,\ t \in BH
$$

In [26]:
#Funciona No solapamiento de aulas
if hasattr(model, 'Restriccion_Aula_Unica'):
    model.del_component(model.Restriccion_Aula_Unica)

# Precomputar las combinaciones de aula, edificio, día y hora
solapamientos = {}

for idx in model.X:
    aula = idx[AULA]
    edificio = idx[EDIFICIO]
    dia = idx[DIA]
    hora = idx[HORA]
    
    if (aula, edificio, dia, hora) not in solapamientos:
        solapamientos[(aula, edificio, dia, hora)] = []

    solapamientos[(aula, edificio, dia, hora)].append(model.X[idx])

# Restricción para el no solapamiento de aulas
def no_solapamiento_aula_rule(model, aula_id, edificio_id, dia, hora):
    if (aula_id, edificio_id, dia, hora) not in solapamientos:
        return pyo.Constraint.Skip  # Si no existe ninguna asignación para esa combinación

    exprs = solapamientos[(aula_id, edificio_id, dia, hora)]
    return sum(exprs) <= 1

# Aplicamos la restricción optimizada
model.Restriccion_Aula_Unica = pyo.Constraint(
    model.Aulas, model.BloquesHorarios,
    rule=no_solapamiento_aula_rule
)

In [27]:
#intento de funcion para obligar a que una materia tenga siempre el mismo aula en la misma hora, modelo no encontraba solucion, proximo intento sera en objetivo



# # Restricción para asegurar que una materia tenga siempre el mismo aula en la misma hora
# def aula_fija_para_primera_hora_rule(model, profesor_id, materia_id, aula_id, edificio_id, dia, hora):
#     if materia_id not in primeras_asignaciones:
#         return pyo.Constraint.Skip
    
#     if (aula_id, dia, hora) != primeras_asignaciones[materia_id]:
#         return pyo.Constraint.Skip

#     return model.X[profesor_id, materia_id, aula_id, edificio_id, dia, hora] == 1


# model.Restriccion_Aula_Fija = pyo.Constraint(
#     model.X.index_set(),
#     rule=aula_fija_para_primera_hora_rule
# )


- El profesor $i$ debe impartir al menos $\gamma_{min}(i)$ horas a la semana y no mas de $\gamma_{max}(i)$ horas a la semana:
$$
\gamma_{min}(i) \leq 
\sum_{j \in M} \sum_{k \in A} \sum_{t \in BH} x^{i, j}_{k, t}
\leq \gamma_{max}(i),\ i \in P
$$

In [28]:
# Eliminar la restricción previa si ya existía
if hasattr(model, 'Restriccion_Min_Horas_Profesor'):
    model.del_component(model.Restriccion_Min_Horas_Profesor)

if hasattr(model, 'Restriccion_Max_Horas_Profesor'):
    model.del_component(model.Restriccion_Max_Horas_Profesor)

# def min_horas_profesor_rule(model, i):
#     exprs = [model.X[idx] for idx in X_index if idx[PROFESOR] == i]
#     if not exprs:
#         return pyo.Constraint.Skip
#     return (model.gamma_min[i] <= sum(exprs))
    
def max_horas_profesor_rule(model, i):
    exprs = [model.X[idx] for idx in X_index if idx[PROFESOR] == i]
    if not exprs:
        return pyo.Constraint.Skip
    return (sum(exprs) <= model.gamma_max[i])

# model.Restriccion_Min_Horas_Profesor = pyo.Constraint(
#     model.Profesores, rule=min_horas_profesor_rule
# )

model.Restriccion_Max_Horas_Profesor = pyo.Constraint(
    model.Profesores, rule=max_horas_profesor_rule
)

- No Solapamiento Profesor (No asignar un profesor a más de una materia en un bloque horario):
$$
\sum_{j \in M} \sum_{k \in A} x^{i, j}_{k, t} \le 1 \quad \forall\ i \in P,\ t \in BH

In [29]:
#Funciona Ningún profesor puede impartir más de una materia en el mismo bloque horario (Falta checar con mas datos)
if hasattr(model, 'Restriccion_Profesor_Unico'):
    model.del_component(model.Restriccion_Profesor_Unico)


# Precomputar los solapamientos de profesores por día y hora
solapamientos_profesor = {}

for idx in model.X:
    profesor = idx[PROFESOR]
    dia = idx[DIA]
    hora = idx[HORA]
    
    if (profesor, dia, hora) not in solapamientos_profesor:
        solapamientos_profesor[(profesor, dia, hora)] = []

    solapamientos_profesor[(profesor, dia, hora)].append(model.X[idx])

def no_solapamiento_profesor_rule(model, p, d, h):
    if (p, d, h) not in solapamientos_profesor:
        return pyo.Constraint.Skip
    exprs = solapamientos_profesor[(p, d, h)]
    return sum(exprs) <= 1

model.Restriccion_Profesor_Unico = pyo.Constraint(
    model.Profesores, model.Dias, datos['Hora'],
    rule=no_solapamiento_profesor_rule
)


- Profesor Puede Enseñar (Asignar un profesor a una materia solo si el profesor puede enseñar esa materia):
$$
x^{i, j}_{k, t} \le \omega(i,j) \quad \forall\ i \in P,\ j \in M,\ k \in A,\ t \in BH
$$

In [30]:
#Funciona Un profesor solo puede impartir una materia 
if hasattr(model, 'Restriccion_Profesor_Compatible'):
    model.del_component(model.Restriccion_Profesor_Compatible)

# Precomputar la relación de profesor y materia
profesor_materia_relacion = {}

for idx in model.X:
    profesor = idx[PROFESOR]
    materia = idx[MATERIA]
    
    if (profesor, materia) not in profesor_materia_relacion:
        profesor_materia_relacion[(profesor, materia)] = model.omega_prof_materia[profesor, materia]

def profesor_puede_ensenar_rule(model, p, m, id_aula, id_edificio, dia, hora):
    return model.X[p, m, id_aula, id_edificio, dia, hora] <= profesor_materia_relacion.get((p, m), 0)

model.Restriccion_Profesor_Compatible = pyo.Constraint(
    model.X.index_set(),
    rule=profesor_puede_ensenar_rule
)


- Aula Compatible Materia (Asignar un aula a una materia solo si el tipo de aula es compatible con el tipo de materia):

$$
x^{i, j}_{k, t} \le \omega(j,k) \quad \forall\ i \in P,\ j \in M,\ k \in A,\ t \in BH
$$

In [31]:
#Funciona Restricción de compatibilidad entre aula y materia (checar con mas datos)
if hasattr(model, 'Restriccion_Aula_Compatible'):
    model.del_component(model.Restriccion_Aula_Compatible)

# Precomputar la compatibilidad entre aula y materia
aula_materia_compatibilidad = {}

for idx in model.X:
    aula = (idx[AULA], idx[EDIFICIO])
    materia = idx[MATERIA]
    
    if (aula, materia) not in aula_materia_compatibilidad:
        aula_materia_compatibilidad[(aula, materia)] = model.omega_aula_materia[aula, materia]

def aula_compatible_materia_rule(model, p, m, aula_id, edificio_id, dia, hora):
    aula = (aula_id, edificio_id)
    return model.X[p, m, aula, (dia, hora)] <= aula_materia_compatibilidad.get((aula, m), 0)

model.Restriccion_Aula_Compatible = pyo.Constraint(
    model.X.index_set(),
    rule=aula_compatible_materia_rule
)

- Si hay una asignacion para una clase a una hora de un maestro, que la materia este

$$
x^{i,j}_{k,t} \le v_{j,k} \quad i \in P, j \in M, k \in A, t \in BH
$$

In [32]:
#Funciona Relación entre X y V (aula asignada a materia) Se hizo bidirectional para que se pueda usar en ambos sentidos
if hasattr(model, 'Relacion_X_V_1'):
    model.del_component(model.Relacion_X_V_1)
if hasattr(model, 'Relacion_X_V_2'):
    model.del_component(model.Relacion_X_V_2)
# Precomputación de compatibilidad entre X y V para todos los posibles (aula, materia)
aula_materia_v = {}

for idx in model.X:
    aula = (idx[AULA], idx[EDIFICIO])
    materia = idx[MATERIA]
    
    if (aula, materia) not in aula_materia_v:
        aula_materia_v[(aula, materia)] = model.V[aula, materia]

def relacion_X_V_rule_1(model, p, m, aula_id, edificio_id, dia, hora):
    aula = (aula_id, edificio_id)
    bloque_horario = (dia, hora)
    
    # Restricción: Si X(p, m, aula, bloque_horario) = 1, entonces V(aula, m) = 1
    return model.X[p, m, aula, bloque_horario] <= aula_materia_v.get((aula, m), 0)

def relacion_X_V_rule_2(model, p, m, aula_id, edificio_id, dia, hora):
    aula = (aula_id, edificio_id)
    bloque_horario = (dia, hora)
    
    # Restricción: Si Z(aula, m) = 0, entonces X(p, m, aula, bloque_horario) = 0
    return model.V[aula, m] >= model.X[p, m, aula, bloque_horario]

model.Relacion_X_V_1 = pyo.Constraint(X_index, rule=relacion_X_V_rule_1)
model.Relacion_X_V_2 = pyo.Constraint(X_index, rule=relacion_X_V_rule_2)



- Las sesiones del curso $j$ deben tener una duracion de al menos $\psi_{min}(j)$ hrs y no mas de
$\psi_{max}(j)$ hrs:
$$
\psi_{min}(j) \cdot z_{j, d} \le
\sum_{i \in P} \sum_{k \in A} \sum_{h \in H} x^{i, j}_{k, (d, h)} \le
\psi_{max}(j) \cdot z_{j, d} \\[0.5em]
j \in M,\ d \in D
$$

In [33]:
if hasattr(model, 'Restriccion_Duracion_Sesion'):
    model.del_component(model.Restriccion_Duracion_Sesion)

if hasattr(model, 'X_jd'):
    model.del_component(model.X_jd)

def X_jd_init(model, j, d):
    return [
        x for x in model.X
        if x[MATERIA] == j and x[DIA] == d
    ]

model.X_jd = pyo.Set(model.Materias, model.Dias, initialize=X_jd_init)

def min_duracion_rule(model, j, d):
    if len(model.X_jd[j, d]) == 0:
        return pyo.Constraint.Skip
    duracion_sesion = sum(model.X[x] for x in model.X_jd[j, d])
    return model.Psi_Min[j] * model.Z[j, d] <= duracion_sesion

def max_duracion_rule(model, j, d):
    if len(model.X_jd[j, d]) == 0:
        return pyo.Constraint.Skip
    duracion_sesion = sum(model.X[x] for x in model.X_jd[j, d])
    return duracion_sesion <= model.Psi_Max[j] * model.Z[j, d]

model.Min_Duracion_Sesion = pyo.Constraint(model.Materias, model.Dias, rule=min_duracion_rule)
model.Max_Duracion_Sesion = pyo.Constraint(model.Materias, model.Dias, rule=max_duracion_rule)

- Solo haya una sesion por dia para una materia $j$ en un dia $d$
$$
\sum_{t \in T} s_{j, t, d} \le z_{j, d} \quad j \in M,\ d \in D
$$

In [ ]:
# #No funciona no idea como arreglarla, 

if hasattr(model, 'Restriccion_Sesion_Diaria'):
    model.del_component(model.Restriccion_Sesion_Diaria)

def una_sesion_por_dia_rule(model, j, d):
    return sum(
        model.S[ids]
        for ids in model.S
        if ids[0] == j
        and ids[2] == d
    ) <= model.Z[j, d]

model.Restriccion_Sesion_Diaria = pyo.Constraint(datos['id_Materias'], datos['Dia'], rule=una_sesion_por_dia_rule)

In [35]:
# def restriccion_consecutiva_o_no_consecutiva_rule(model, m, d):
#     # Verificar si 'm' está en datos antes de acceder a las horas
#     if m not in datos['Horas_Semana_Materia']:
#         raise ValueError(f"La materia {m} no se encuentra en el diccionario de datos.")
    
#     # Obtener las horas asignadas para la materia m en el día d
#     horas_asignadas = [h for h in datos['Hora'] if model.S[m, h, d].value == 1]
    
#     # Obtener el número de horas semanales de la materia
#     horas_semanales = datos['Horas_Semana_Materia'][m]

#     # Si la materia tiene menos de 4 horas semanales, se puede asignar de forma corrida
#     if horas_semanales < 4:
#         # Asegurarse de que todas las horas estén asignadas de forma corrida en un solo día
#         if len(horas_asignadas) > 1:
#             horas_asignadas.sort()
#             # Verificar si las horas son consecutivas
#             for i in range(1, len(horas_asignadas)):
#                 if horas_asignadas[i] != horas_asignadas[i-1] + 1:
#                     return pyo.Constraint.Infeasible  # Si no son consecutivas, es inviable
#         return sum(model.S[m, h, d] for h in datos['Hora']) == horas_semanales  # Verificar si la asignación es correcta

#     # Si la materia tiene 4 o más horas semanales, verificamos la asignación en días separados
#     if horas_semanales >= 4:
#         # Verificamos que no haya más de 2 horas asignadas en el mismo día
#         if len(horas_asignadas) > 2:
#             return pyo.Constraint.Infeasible  # Más de 2 horas en un día no es permitido

#         # Asegurarnos de que las horas asignadas sean consecutivas si están en un solo día
#         horas_asignadas.sort()
#         consecutive_hours = 1

#         # Recorremos todas las horas asignadas y verificamos las consecutivas
#         for i in range(1, len(horas_asignadas)):
#             if horas_asignadas[i] == horas_asignadas[i-1] + 1:
#                 consecutive_hours += 1
#             else:
#                 consecutive_hours = 1  # Si no son consecutivas, restablecemos el contador

#             # Si las horas consecutivas son más de 2, entonces es inviable
#             if consecutive_hours > 2:
#                 return pyo.Constraint.Infeasible

#     # **Distribuir las horas de manera que no se asignen más de 2 horas en un solo día**
#     if horas_semanales >= 4 and len(horas_asignadas) > 2:
#         return pyo.Constraint.Infeasible  # Más de 2 horas en un solo día no es permitido

#     # Si la materia tiene horas no consecutivas, deben ser asignadas en diferentes días
#     if horas_semanales >= 4 and len(horas_asignadas) > 1:
#         dias_asignados = set()  # Usamos un set para almacenar los días asignados
#         for h in horas_asignadas:
#             dia = d  # Ya tenemos el día (d) en el que estamos iterando
#             if dia in dias_asignados:
#                 return pyo.Constraint.Infeasible  # No puede repetirse el día
#             dias_asignados.add(dia)

#     # Asegurarse de que en cada día solo haya una clase asignada por materia
#     return sum(model.S[m, h, d] for h in datos['Hora']) <= 1  # Cada materia debe estar asignada a una sola hora por día

# model.restriccion_consecutiva_o_no_consecutiva = pyo.Constraint(datos['id_Materias'], datos['Dia'], rule=restriccion_consecutiva_o_no_consecutiva_rule)


In [36]:
# def restriccion_distrubucion_horas(model, m):
#     # Obtener las horas asignadas para la materia m en días específicos
#     horas_asignadas = [
#         h for h in datos['Hora'] for d in datos['Dia'] if model.S[m, h, d].value == 1
#     ]
    
#     # Obtener el número de horas semanales de la materia
#     horas_semanales = datos['Horas_Semana_Materia'][m]

#     # Si la materia tiene más de 4 horas semanales, verificamos distribución
#     if horas_semanales >= 4:
#         dias_asignados = set()
        
#         # Recorremos las horas asignadas y asignamos los días correspondientes
#         for h in horas_asignadas:
#             for d in datos['Dia']:  # Revisamos todos los días posibles
#                 if model.S[m, h, d].value == 1:
#                     dias_asignados.add(d)

#         # Asegurarse de que no haya más de 2 días asignados
#         if len(dias_asignados) > 2:
#             return pyo.Constraint.Infeasible  # No más de 2 días para materias con 4 o más horas

#         # Verificar que las horas asignadas sean consecutivas en cada día
#         for d in dias_asignados:
#             horas_en_dia = [h for h in horas_asignadas if model.S[m, h, d].value == 1]
#             horas_en_dia.sort()  # Ordenamos las horas en el día

#             # Verificar que las horas sean consecutivas (sin gaps)
#             for i in range(1, len(horas_en_dia)):
#                 if horas_en_dia[i] != horas_en_dia[i-1] + 1:
#                     return pyo.Constraint.Infeasible  # Si no son consecutivas, es inviable

#     return pyo.Constraint.Feasible
    
# model.restriccion_distrubucion_horas = pyo.Constraint(datos['id_Materias'], rule=restriccion_distrubucion_horas)


- Restriccion si una materia inicia en una hora $t$, que la variable $s$ marque su inicio
$$
s_{j, h, d} \ge \sum_{k \in A} x^{i, j}_{k, t} - \sum_{k \in A} x^{i, j}_{k, t'} \\[0.5em]
t' = (d, t -1),\ i \in P,\ j \in M,\ t \in H\setminus 1,\ d \in D
$$

In [37]:
if hasattr(model, 'Restriccion_Var_S'):
    model.del_component(model.Restriccion_Var_S)

X_by_prof_materia_day_hour = {}
for key in model.X:
    i, j, _, _, d, h = key
    X_by_prof_materia_day_hour.setdefault((i, j, d, h), []).append(key)

def var_s_rule(model, i, j, h, d):
    keys_curr = X_by_prof_materia_day_hour.get((i, j, d, h), [])
    keys_prev = X_by_prof_materia_day_hour.get((i, j, d, h-1), [])

    if not keys_curr and not keys_prev:
        return pyo.Constraint.Skip  # skip if no relevant variables

    return model.S[j, h, d] >= sum(model.X[k] for k in keys_curr) - sum(model.X[k] for k in keys_prev)

model.Restriccion_Var_S = pyo.Constraint(model.Profesores, model.Materias, datos['Hora'][1:], model.Dias, rule=var_s_rule)

In [38]:
#Modelo V1 Duracion : 30 segundos

# =============================================
# FUNCIÓN OBJETIVO REESCRITA Y OPTIMIZADA
# =============================================

# ------------------------
# Parámetros de ponderación
# ------------------------
peso_profesores_activos = -1.0      # Maximizar uso de profesores reales
peso_aulas = -1                   # Maximizar cantidad de aulas diferentes usadas
peso_asignaciones = -1            # Maximizar densidad horaria (más clases asignadas)
peso_distribucion = -1            # Maximizar distribución uniforme (menos varianza por día)
peso_aula_inconsistente = 20.0      # Penalizar cambios de aula por materia
peso_hora_inconsistente = 20.0       # Penalizar cambios de hora por materia
M = 1000                           # Penalización fuerte para profesores imaginarios


# ------------------------
# Precomputación
# ------------------------

# Profesores válidos según su tipo
profesores_reales_validos = [
    (p, m) for (p, m) in model.Y if p in datos['Id_Profesores']
]
profesores_imaginarios_validos = [
    (p, m) for (p, m) in model.Y if p in datos['Profesores_Imaginarios']
]

# Índices por profesor y día para evaluar dispersión horaria
distribucion_indices = {
    (p, d): [
        (p, m, a, (d, h)) for m in model.Materias for a in model.Aulas for h in datos['Hora']
        if (p, m, a, (d, h)) in model.X
    ]
    for p in datos['Profesores_Totales'] for d in datos['Dia']
}

# ------------------------
# Asignación inicial de aula/día/hora por materia (solo una vez)
# ------------------------

primeras_asignaciones = {}
for idx in model.X:
    materia = idx[MATERIA]
    aula = idx[AULA]
    dia = idx[DIA]
    hora = idx[HORA]
    if materia not in primeras_asignaciones:
        primeras_asignaciones[materia] = (aula, dia, hora)

# ------------------------
# Definición de la función objetivo
# ------------------------

def objetivo_optimo(model):
    # Profesores reales utilizados
    profesores_reales_utilizados = sum(model.Y[p, m] for (p, m) in profesores_reales_validos)

    # Profesores imaginarios utilizados
    profesores_imaginarios_utilizados = sum(model.Y[p, m] for (p, m) in profesores_imaginarios_validos)

    # Aulas distintas utilizadas
    aulas_utilizadas = sum(model.V[a, m, d] for (a, m, d) in model.V)

    # Asignaciones totales de clases
    asignaciones_totales = sum(model.X[idx] for idx in model.X.index_set())

    # Distribución diaria de cada profesor
    distribucion = 0
    for p in datos['Profesores_Totales']:
        horas_por_dia = {
            d: sum(model.X[idx] for idx in distribucion_indices.get((p, d), []))
            for d in datos['Dia']
        }
        media = sum(horas_por_dia.values()) / len(datos['Dia'])
        varianza = sum((h - media) ** 2 for h in horas_por_dia.values()) / len(datos['Dia'])
        distribucion += varianza

    # Penalización por materia que cambia de aula o horario
    penalizacion_aula_fija = sum(
        model.X[p, m, a, e, d, h]
        for (p, m, a, e, d, h) in model.X.index_set()
        if m in primeras_asignaciones and a != primeras_asignaciones[m][0]
    )

    # Hora inconsistente
    penalizacion_hora_fija = sum(
        model.X[p, m, a, e, d, h]
        for (p, m, a, e, d, h) in model.X.index_set()
        if m in primeras_asignaciones and h != primeras_asignaciones[m][2]
    )

    # Función objetivo final
    return (
        peso_profesores_activos * profesores_reales_utilizados +
        peso_aulas * aulas_utilizadas +
        peso_asignaciones * asignaciones_totales +
        peso_distribucion * distribucion +
        M * profesores_imaginarios_utilizados +
        peso_aula_inconsistente * penalizacion_aula_fija +
        peso_hora_inconsistente * penalizacion_hora_fija
    )

# ------------------------
# Aplicar al modelo
# ------------------------

if hasattr(model, 'Objetivo'):
    model.del_component('Objetivo')

model.Objetivo = pyo.Objective( 
    rule=objetivo_optimo,
    sense=pyo.minimize
)


In [39]:
def resolver_modelo(model, mipgap=0.02, threads=4, emphasis=1):
    from pyomo.opt import SolverFactory, TerminationCondition

    solver = SolverFactory('cplex')

    if not solver.available():
        raise RuntimeError("CPLEX no está disponible. Verifica la instalación o el PATH.")

    resultado = solver.solve(model, tee=True)

    # Verificar si la solución fue óptima o factible
    condicion = resultado.solver.termination_condition
    if condicion in [TerminationCondition.optimal, TerminationCondition.feasible]:
        print(f"Solución encontrada: {condicion}")
    else:
        print(f"Problema durante la resolución: {condicion}")
        print("Revisa el modelo o ajusta parámetros del solver.")

    return resultado

In [40]:
def mostrar_bloque(dia, hora):
    dias = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes']
    dia_str = dias[dia - 1]  # porque 1 = Lunes
    hora_inicio = 7 + hora  # si hora=1 corresponde a 8am
    hora_fin = hora_inicio + 1
    return f"{dia_str} {hora_inicio:02d}:00 - {hora_fin:02d}:00"

def mostrar_asignacion(model):
    asignaciones = []  # lista para guardar los datos

    print("\nAsignaciones de profesores, materias, aulas y horarios:")
    for idx in model.X:
        if pyo.value(model.X[idx]) > 0.5:
            p, m, aula, edificio, dia, hora = idx
            profesor_nombre = datos['nombres_profesor'].get(p, p)
            materia_nombre = datos['nombres_materia'].get(m, m)
            dia_str = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes'][dia - 1]
            hora_inicio = 7 + hora
            hora_fin = hora_inicio + 1
            hora_str = f"{hora_inicio:02d}:00 - {hora_fin:02d}:00"
            
            print(f"Profesor: {profesor_nombre} | Materia: {materia_nombre} | "
                  f"Aula: {aula} (Edificio {edificio}) | Horario: {dia_str} {hora_str}")
            
            asignaciones.append({
                'Profesor': profesor_nombre,
                'Materia': materia_nombre,
                'Aula': aula,
                'Edificio': edificio,
                'Día': dia_str,
                'HoraInicio': f"{hora_inicio:02d}:00",
                'HoraFin': f"{hora_fin:02d}:00",
                'HoraNum': hora,  # útil para ordenamientos
                'DiaNum': dia,    # útil para ordenamientos
                'AulaCompleta': f"{aula} (Edificio {edificio})"
            })

    return asignaciones

In [41]:
# Resolver el modelo
resultados = resolver_modelo(model)

# Verificar si encontró solución
if (resultados.solver.status == pyo.SolverStatus.ok and
    resultados.solver.termination_condition == TerminationCondition.optimal):

    print("Solución óptima encontrada.\n")
    asignaciones_lista = mostrar_asignacion(model)

    import pandas as pd
    df_Una_Asignacion_por_Dia = pd.DataFrame(asignaciones_lista)

    # ✅ Ahora ya puedes verificar solapamientos
    # # Por ejemplo: por profesor + día + hora
    # solapes_profesor = df_asignaciones[df_asignaciones.duplicated(['Profesor', 'Día', 'HoraInicio'], keep=False)]
    # solapes_aula = df_asignaciones[df_asignaciones.duplicated(['AulaCompleta', 'Día', 'HoraInicio'], keep=False)]

    # if solapes_profesor.empty:
    #     print("✅ No hay solapamientos de profesor.")
    # else:
    #     print("⚠️ Hay solapamientos de profesor:")

    # if solapes_aula.empty:
    #     print("✅ No hay solapamientos de aula.")
    # else:
    #     print("⚠️ Hay solapamientos de aula:")

else:
    print("No se encontró solución óptima.")
    print("Estado:", resultados.solver.status)
    print("Condición de terminación:", resultados.solver.termination_condition)


Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer 22.1.1.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2022.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a command name for more
information on commands.

CPLEX> Logfile 'cplex.log' closed.
Logfile '/tmp/tmpwexz_hwa.cplex.log' open.
CPLEX> Problem '/tmp/tmpefo9a11t.pyomo.lp' read.
Read time = 2.23 sec. (86.75 ticks)
CPLEX> Problem name         : /tmp/tmpefo9a11t.pyomo.lp
Objective sense      : Minimize
Variables            :  206197  [Binary: 206197]
Objective nonzeros   :  203447
Linear constraints   :  811299  [Less: 811186,  Equal: 113]
  Nonzeros           : 2769661
  RHS nonzeros       :  407151

Variables            : Min LB: 0.000000         Max UB: 1.000000       
Objective nonzeros   : Min   : 1.000000         Max   : 1000.000       
Linear constraints   :
  Nonzeros           : 

In [44]:
df_Una_Asignacion_por_Dia.to_csv("horario.csv")

In [43]:
df_old[:40]

NameError: name 'df_old' is not defined